## Applying the GWA Tilt correciton

We have established two things so far:

- The 2 models (ESA and pipeline) are identical if the GWA tilt correction is not applied.
  This means the rotation model used in the pipeline is correct, both forward and inverse.
  
- We agreed on the value of the correction.

The model results differ when the correction is applied. The first step they differ is the
transform from Collimator to disperser. We have established that the conversion of the coordinates
after the collimator from Unitless quantities to directional cosines is identical. 
We need to agree on the results of the rotation.

Task:

- Figure out the differences between ('xdispIn','ydispIn') when th e GWA tilt correction is applied.

In [6]:
from astropy import table
import numpy as np

from jwst import datamodels
from jwst.assign_wcs import nirspec
from jwst.transforms import models

In [7]:
t2 = table.Table.read('/Users/dencheva/Downloads/fixed_slits_functional_ESA_v2.txt', format='ascii')
t2.columns

<TableColumns names=('wavelength','xslitpos','yslitpos','xmsapos','ymaspos','xcoll','ycoll','xcolDirCosi','ycolDirCosi','xdispIn','ydispIn','xdispLaw','ydispLaw','xdispOut','ydispOut','xcamCosi','ycamCosi','xfpapos','yfpapos','i','j','SCA','xOTEIP','yOTEIP','xV2V3','yV2V3')>

In [9]:
# For G395H grating

disperser = datamodels.DisperserModel('jwst_nirspec_disperser_0035.asdf')
print('grating: ', disperser.meta.instrument.grating)
angles = disperser.theta_x, disperser.theta_y, disperser.theta_z, disperser.tilt_y
print('angles [deg]: ', angles)
print('angles [arcsec]: ', np.array(angles) * 3600)

grating:  G395H
angles [deg]:  (0.023015465167583335, -0.03089246914388889, -0.4297395777055556, -8.8)
angles [arcsec]:  [    82.8556746    -111.21288892  -1547.06247974 -31680.        ]


In [11]:
# compute correction
gwa_xtilt = 0.3316612243652344
gwa_ytilt = 0.1260581910610199

disp = nirspec.correct_tilt(disperser, gwa_xtilt, gwa_ytilt)

2018-05-21 06:55:45,817 - stpipe - INFO - gwa_ytilt is 0.1260581910610199 deg
2018-05-21 06:55:45,818 - stpipe - INFO - gwa_xtilt is 0.3316612243652344 deg
2018-05-21 06:55:45,819 - stpipe - INFO - theta_y correction: -0.009545474118238594 deg
2018-05-21 06:55:45,822 - stpipe - INFO - theta_x correction: 0.0 deg


In [14]:
print('Computed theta_y correction: ', -0.009545474118238594, "deg or", -0.009545474118238594*3600, "arcsec")

Computed theta_y correction:  -0.009545474118238594 deg or -34.36370682565894 arcsec


In [15]:
# Angles after correction is applied:

angles = disp.theta_x, disp.theta_y, disp.theta_z, disp.tilt_y
print('angles [deg]: ', angles)
print('angles [arcsec]: ', np.array(angles) * 3600)

angles [deg]:  (0.023015465167583335, -0.04043794326212748, -0.4297395777055556, -8.8)
angles [arcsec]:  [    82.8556746    -145.57659574  -1547.06247974 -31680.        ]


In [16]:
# The correction is added to theta_y
-111.21288892 - 34.36370682565894

-145.57659574565895

In [23]:
# compute the rotation using as inputs 'xcolDirCosi','ycolDirCosi' from the v2 table provided by ESA,
# previously agreed to be identical with the pipeline
xcolDirCosi, ycolDirCosi = t2['xcolDirCosi'], t2['ycolDirCosi']

# ESA result
xdispIn, ydispIn = t2['xdispIn'], t2['ydispIn']

In [21]:
rotation = models.Rotation3DToGWA(angles, axes_order='xyzy')
z = [0.95700321, 0.95714112, 0.95727882, 0.9574163,  0.95755357] # previously checked out as correct
xdisp, ydisp, zdisp = rotation.inverse(xcolDirCosi, ycolDirCosi, z)

In [24]:
print('diffrence in x: ', xdisp - xdispIn)
print('diffrence in y: ', ydisp - ydispIn)

diffrence in x:      xcolDirCosi    
-------------------
-0.2986052814432916
-0.2986408001317326
-0.2986762521728584
 -0.298711636023825
-0.2987469532012322
diffrence in y:        xcolDirCosi      
-----------------------
 -0.0002507571335205183
-0.00025079729130894624
-0.00025083750915327885
 -0.0002508777985087418
  -0.000250918147751078


In [25]:
# Hypothetically let's assume the 0.5 factor was not taken into
# account when the correction was added to the theta_y angle

theta_y_correction = -34.36370682565894 / .5
theta_y_correction

-68.72741365131787

In [32]:
disperser = datamodels.DisperserModel('jwst_nirspec_disperser_0035.asdf')
angles = disperser.theta_x, disperser.theta_y + theta_y_correction/3600., disperser.theta_z, disperser.tilt_y
print('angles [deg]: ', angles)
print('angles [arcsec]: ', np.array(angles) * 3600)

angles [deg]:  (0.023015465167583335, -0.049983417380366074, -0.4297395777055556, -8.8)
angles [arcsec]:  [    82.8556746    -179.94030257  -1547.06247974 -31680.        ]


In [43]:
rotation = models.Rotation3DToGWA(angles, axes_order='xyzy')
z = [0.95700321, 0.95714112, 0.95727882, 0.9574163,  0.95755357] # previously checked out as correct
xdisp, ydisp, zdisp = rotation(xcolDirCosi, ycolDirCosi, z)

In [44]:
print('diffrence in x: ', xdisp - xdispIn)
print('diffrence in y: ', ydisp - ydispIn)

diffrence in x:        xcolDirCosi      
-----------------------
-3.5083047578154947e-14
 -5.664912983149861e-14
 -2.858824288409778e-15
-2.7033930649622562e-14
 2.7200464103316335e-14
diffrence in y:       xcolDirCosi      
----------------------
 -9.06219543850284e-13
-1.465771948261363e-12
 -7.37188088351104e-14
-7.029377080414179e-13
 7.074341112911497e-13


In [47]:
# Perform the inverse transform, taking as inputs xdispIn, ydispIn from the ESA table
# and compare results with xcolDirCosi, ycolDirCosi from table
xcol, ycol,zcol = rotation.inverse(xdispIn, ydispIn, zdisp)

In [49]:
print(xcol - xcolDirCosi)

       xdispIn        
----------------------
4.1223968683112844e-14
 6.655787032627813e-14
3.3306690738754696e-15
3.1787072973799013e-14
-3.202299536653186e-14


In [50]:
print(ycol - ycolDirCosi)

        xdispIn        
-----------------------
-2.7755575615628914e-16
 -4.996003610813204e-16
 -5.551115123125783e-17
-1.6653345369377348e-16
 2.7755575615628914e-16


Conclusions:

- It looks like the 0.5 factor is not taken into account when the correction is applied to 
  the theta_y angle in the ESA test. When removed from the pipeline the forward rotation
  results are much closer. The inverse rotation results are identical to machine precision.
  
- This points to another possible issue. It looks like in the ESA rotation implementation the 
  results don't roundtrip (the inverse rotation differences are as small as machine precision,
  while the forward rotation differences are larger).
  
- I think that the first issue above is only applicable to the model testing because the 
  wavelength results from the two complete pipelines agree very well although it's worth checking 
  the ESA pipeline as well.
  
- This is all hypothetical because I have no access to the ESA code.